# Company Agent with UiPath

## Get insights. Take action - with your Enterprise-Grounded Agent

The real value of AI agents emerges in the **workplace**, where context, reliability, and impact matter most. 

Every company is different — with unique processes, policies, tools, and requirements. 

For AI to be truly useful, **an agent should be part of the company itself**: rooted in enterprise context, connected to its systems, and able to turn data into reliable insights and trusted actions.

<p align="center"><img src="resources/Company_Agent_not_highlighted.png" alt="Company Agent" width="800"></p>
 

# Company Policy Agent
## Keep the Policies, Lose the Hassle
Let’s be honest—company policies matter, and they’re here to stay.

Still, everyday questions often create friction:
- What’s the standard laptop I can have?
- How do I expense $10 of API calls?
- Does this type of software use comply with company rules?

A dedicated, contextualized, secure agent lets employees ask about policy and take compliant action—fast. Here’s how the experience looks and how to implement it with UiPath.

<p align="center"><img src="resources/Company_Agent_highlighted.png" alt="Company Agent" width="800"></p>

# Build, Deploy & Use the Company Policy Agent with UiPath

**Solution & Steps**

1. **Define the Company Policy Reference**

2. **Create UiPath Context Grounding Index from the policy text**

3. **Initialize the project**

4. **Build and Publish the Company Policy Agent**
   1. Visualize the agent  
   2. Add the `input.json` file with a test input  
   3. Run locally  
   4. Pack & Publish the Agent to UiPath Platform  

5. **Deploy the Coded Agent as a process within UiPath Platform**





### Step 1. Define the Company Policy Reference 
<svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 100 100">
  <rect width="100" height="100" fill="#FF6A00" rx="15"/>
  <text x="50%" y="58%" text-anchor="middle" font-size="50" font-family="Arial, sans-serif" fill="white">Ui</text>
</svg> 

Write the company policies you want the Agent to be aware of and save them in a plain txt file (e.g., [company_policy.txt](company_policy.txt)) in the current directory.

**Alternatively, copy the sample Company Policy text - this will be created in your current directory**

In [ ]:
import shutil

shutil.copy("resources/company_policy.txt", "company_policy.txt")

### Step 2. Create UiPath Context Grounding Index from the policy text

<svg xmlns="http://www.w3.org/2000/svg" width="20" height="20" viewBox="0 0 100 100">
  <rect width="100" height="100" fill="#FF6A00" rx="15"/>
  <text x="50%" y="58%" text-anchor="middle" font-size="50" font-family="Arial, sans-serif" fill="white">Ui</text>
</svg> 

Create Context Grounding Index from the policy text
In your UiPath platform account, in Orchestrator create a new Storage Bucket and upload the [company_policy.txt](company_policy.txt) file to it



<p align="center"><img src="resources/UiPath_Storage_Buckets.png" alt="UiPath Storage Buckets" width="850"></p>

---
<p align="center"><img src="resources/UiPath_Storage_Buckets_Add_File.png" alt="UiPath Storage Buckets Add File" width="850"></p>

With this, one can create a context grounding index based on the company_policy.txtby using Storage Bucket as data source. 

<p align="center"><img src="resources/UiPath_Create_Index.png" alt="UiPath Create Index" width="850"></p>


### Step 3. Initialize the project 

⚠️ **Not needed if "Hello World example was ran before, in the same project"**


Set the envornment

In [ ]:
!py -m venv .venv
!.venv\Scripts\pip install ipykernel jupyter
!.venv\Scripts\python -m ipykernel install --user --name=jupyter-uv-env --display-name "WellsFargoLab"

In [ ]:
# Install the uipath package and upgrade it
%pip install --upgrade uipath-langchain

In [ ]:
# Verify the uipath installation using pip
%pip show uipath-langchain

In [ ]:
#Initialize the UiPath LangChain agent. replace your name here
!uipath new companyAgent-<<< ADD YOUR NAME HERE >>>

### Step 3.1 Authenticate with UiPath Platform

⚠️ **Not needed if "Hello World example was ran before, in the same project"**


In [ ]:
# Authenticate to UiPath Staging. Make sure to specify the name of the target Tenant 
!uipath auth --force --tenant AgenticILT

✏️ Check if the credentials were copied in the [`.env`](./.env) file. It should look somehting like this:

```env
UIPATH_URL=https://cloud.uipath.com/tpenlabs/AgenticILT
UIPATH_TENANT_ID=xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
UIPATH_ORGANIZATION_ID=xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
UIPATH_ACCESS_TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.xxxxxxxxxxxxxxxxxxxxx


### Step 4. Build and Publish the Company Policy Agent

The Coded Agent is built with the LangGraph framework and has a single node called `policy` that based on the user ask and company policy composes response via an LLM call 

Start with defining the System Prompt - should gives the LLM model of choice to the rules, role, and behavior for how it should respond.


In [ ]:
# %%writefile -a main.py

from langgraph.graph import END, START, MessagesState, StateGraph
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel
from uipath_langchain.retrievers import ContextGroundingRetriever
from langchain_core.documents import Document
from uipath.platform.common import InvokeProcess
from uipath.platform.errors import IngestionInProgressException
import httpx
from uipath_langchain.chat.models import UiPathAzureChatOpenAI

class IndexNotFound(Exception):
    pass

# Define system prompt for company policy agent
system_prompt = """
You are an advanced AI assistant specializing in answering questions about internal company policies. 
Structure your responses as follows:

Answer:
<1–3 sentence conclusion>

Policy evidence:
- "<exact quote>"
  Reference: <section/bullet>

Conditions/Exceptions:
- <condition or threshold>

Next steps:
- <action 1>
- <action 2>

If not covered:
<one line stating it’s not covered and what info is missing>
"""

Add a helper method to get the context data from UiPath Platform to be leveraged by the UiPath Coded Agent

In [ ]:
# %%writefile -a main.py

async def get_context_data_async(retriever: ContextGroundingRetriever, question: str) -> list[Document]:
    no_of_retries = 5
    context_data = None
    data_queried = False
    while no_of_retries != 0:
        try:
            context_data = await retriever.ainvoke(question)
            data_queried = True
            break
        except IngestionInProgressException as ex:
            logger.info(ex.message)
            no_of_retries -= 1
            logger.info(f"{no_of_retries} retries left")
            time.sleep(5)
        except httpx.HTTPStatusError as err:
            if err.response.status_code == 404:
                raise IndexNotFound
            raise
    if not data_queried:
        raise Exception("Ingestion is taking too long.")
    return  context_data

In [ ]:
# %%writefile -a main.py

class GraphInput(BaseModel):
    """Input for the company policy agent graph."""
    question: str


class GraphOutput(BaseModel):
    """Output for the company policy agent graph."""
    response: str

async def policy_node(state: GraphInput) -> GraphOutput:
    retriever = ContextGroundingRetriever(
                index_name="company-policy-index",
                number_of_results=100,
            )
    try:
        context_data= await get_context_data_async(retriever, state.question)
    except IndexNotFound:
        context_data = system_prompt
    

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": state.question},
        {"role":"user", "content": context_data[0].page_content}
    ]

    """Process a company policy question and return a structured answer."""
    user_message = f"""Please answer the following company policy question. Use clear section headers for each relevant policy area (Procurement Policy, Equipment Acquisition, PTO Policy).If the question is outside these topics, politely state your scope.\n\nQuestion: {state.question}"""
    # result = await policy_agent.ainvoke(new_state)
    result = await llm.ainvoke(messages)
    return GraphOutput(response=result.content)

**Set the LLM model and its parameters**

> **Recommended: Use Option 2 (UiPathChatOpenAI)** - No API key required, uses your existing UiPath authentication.

> **Available Models:** For the list of available models, refer to the [list of available models (under "agenthub")](https://github.com/UiPath/gitops-centralized-cluster/blob/master/projects/prod/llms/llmgateway-service/llmgateway-service-values.yaml#L2301) or use `sdk.agenthub.get_available_llm_models()` to fetch available models programmatically.

> **Note:** All `resources/mainX.py` files use `ChatOpenAI` by default. To use UiPath LLM instead (no API key needed), change `llm = ChatOpenAI(...)` to `llm = UiPathChatOpenAI(...)` in the respective file before running.

---

**Option 1.** Use generic chat models such as ChatOpenAI. The corresponding API Key needs to be provided and copied in the [`.env`](./.env) file.

```env
OPENAI_API_KEY=sk-proj-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
# TEMPORARY API KEY (FOR TESTING PURPOSES-WILL BE REMOVED AFTER THE SESSION)

with open(".env", "a") as f:
    f.write("\nOPENAI_API_KEY=****\n")
    f.write("UIPATH_FOLDER_PATH=WellsFargoCodedAgents\n")

In [ ]:
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()


llm = ChatOpenAI(
    model="gpt-5",
    temperature=0,
    max_tokens=4000,
    timeout=30,
    max_retries=2,
)

**Option 2 (Recommended).** UiPath provides multiple chat models, for example: `UiPathChatOpenAI` and `UiPathChat`. These are compatible with `LangGraph` as drop-in replacements. 

**No API Key required** - uses your existing UiPath authentication token.

> **Important:** If you want to use this option, you must also update `resources/main1.py` to use `UiPathChatOpenAI` instead of `ChatOpenAI`, since that file gets copied to `main.py` later in the notebook.

In [ ]:
from uipath_langchain.chat import UiPathChatOpenAI

llm = UiPathChatOpenAI(
    model="gpt-4o-2024-08-06",
    temperature=0,
    max_tokens=4000,
    timeout=30,
    max_retries=2,
    # other params...
)

Build the state graph. START -> POLICY NODE -> END (collect output)

In [ ]:
# %%writefile -a main.py

# Build the state graph
builder = StateGraph(GraphInput, output_schema=GraphOutput)
builder.add_node("policy", policy_node)

builder.add_edge(START, "policy")
builder.add_edge("policy", END)

Finally, compile the graph

In [ ]:
# %%writefile -a main.py

# Compile the graph
graph = builder.compile()

In [ ]:
# Copy the above python code to the main.py file

import shutil
shutil.copyfile("resources/main1.py", "main.py")

In [ ]:
# Initialize the project to make sure all file are up to date
!uipath init

----

#### Visualize the agent

👀 View the generated Mermaid diagram: [agent.mermaid](agent.mermaid)

In [ ]:
# See the Mermaid diagram
from IPython.display import display, Markdown

with open("agent.mermaid", "r") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

#### Add the input.json file with a test input

`{
    "question": "Give me the list of standard laptop equipment"
}`

In [ ]:
%%writefile input.json
{
  "question": "What is the PTO policy for taking 3 days off"
}

#### Run and Test Locally

Before deploying the Coded Agent to UiPath Platform, let's test it locally to make sure it has the expected behavior

In [ ]:
# Re-initialize the project to make sure all file are up to date
!uipath init

In [ ]:
# Run the agent locally
!uipath run agent --file input.json

#### 🚀 Pack & Publish the Agent to UiPath Platform

The following steps will **package the Coded Agent** as a UiPath artifact and **publish it at Folder level**.

---

📝 **Pre-Publish Checklist**

Before packing and publishing, make sure your project metadata in [`pyproject.toml`](./pyproject.toml) is correct:

- **Version** → Set the `version` field to your intended release version  
- **Author(s)** → Update the `authors` field as needed  
- **Description** → Ensure it reflects the purpose of the project  
- **Dependencies** → Verify required packages are listed properly  

---

 ✅ Example `pyproject.toml`

```toml
[project]
name = "company-policy"
version = "0.0.10"
description = "company-policy"
authors = [{ name = "Eusebiu Jecan", email = "eusebiu.jecan@uipath.com" }]

dependencies = [
    "uipath-langchain>=0.0.106",
]

requires-python = ">=3.11"


In [ ]:
# Auto-increase the version of the package
import re, pathlib
p = pathlib.Path("pyproject.toml"); t = p.read_text()
n = re.sub(r'(?m)^(version\s*=\s*")(\d+)\.(\d+)\.(\d+)(")', lambda m: f'{m.group(1)}{m.group(2)}.{m.group(3)}.{int(m.group(4))+1}{m.group(5)}', t, count=1)
p.write_text(n); print(re.search(r'(?m)^version\s*=\s*"([^"]+)"', t).group(1), "->", re.search(r'(?m)^version\s*=\s*"([^"]+)"', n).group(1))

In [ ]:
!uipath pack
!uipath publish -f WellsFargoCodedAgents

## Step 5. Deploy/Upgrade the Coded Agent as a process within UiPath Platform 

If the agent was deployed before one just needs to upgrade the version of the process

<p align="center"><img src="resources/UpgradeProcessVersion.png" alt="Upgrade Process Version" width="820"></p>


Once published, the coded Agent appears in the Folder feed as an Agent package.

- **Where to find it**: Orchestrator → WellsFargoCodedAgents Folder → Automations → Folder Packages

<p align="center"><img src="resources/UiPath_Agent_Package_Tenant_Feed_Versions.png" alt="UiPath Agent Package Tenant Feed Versions" width="820"></p>

- **Next step**: Deploy the package as a process and set the environment variables

<p align="center"><img src="resources/UiPath_Add_Process.png" alt="UiPath Add Process" width="820"></p>

<p align="center"><img src="resources/UiPath_Add_Process_Setting.png" alt="UiPath Add Process Setting" width="820"></p>





# Why Coded Agents?

## **Complexity**. Unlocks branching agent logic 

Moves beyond simple contextualized LLM calls

- Advanced trajectories. Multiple nodes, steps and states.  

- Nuanced decision making

----

**Multiple Agent Architecture**

For a company, a single agent cannot address all needs. Instead, organizations require specialized, context-aware, and tailored logic agents—along with a reliable mechanism to route each request to the most appropriate one.

We can extend the single-agent design into a supervisor-dispatch pattern that routes each request to a specialized agent.

- Supervisor node: Classifies the question into an agent label and stores it in `category` in the graph state.
- Routing: A router function sends the request to the right Agent (graph node) using conditional edges.
- Specialized agents:
  - `policy` (Company Policy): existing agent using `company-policy-index`.
  - `procurement`: a new agent (copied from policy) using `procurement-index`.

Key edits in `main.py`:
- Added `GraphState` with fields: `question`, `category`.
- Added `supervisor_node` to classify into “Company Policy” or “Procurement”.
- Added `route_by_category(state)` to return the next node key.
- Added `procurement_node` (initially same prompt shape as policy).
- Rewired graph: `START -> supervisor -> (policy | procurement) -> END`.


---

In [ ]:
# Added GraphState with fields: question, category
from pydantic import BaseModel

class GraphState(BaseModel):
    """State for the company policy agent graph."""
    question: str
    category: str | None = None


In [ ]:
# Added supervisor_node to classify into Company Policy or Procurement
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5", temperature=0, max_tokens=4000, timeout=30, max_retries=2)

async def supervisor_node(state: GraphState) -> GraphState:
    categories = ["Company Policy", "Procurement"]
    messages = [
        {"role": "system", "content": ("Classify the user's question into one of: Company Policy, Procurement. Reply with only the exact label.")},
        {"role": "user", "content": state.question},
    ]
    result = await llm.ainvoke(messages)
    label = (result.content or "").strip()
    normalized = {c.lower(): c for c in categories}
    category = normalized.get(label.lower(), "Company Policy")
    return GraphState(question=state.question, category=category)


In [ ]:
# Added procurement_node (initially same prompt shape as policy)
from uipath_langchain.retrievers import ContextGroundingRetriever

async def procurement_node(state: GraphState):
    retriever = ContextGroundingRetriever(
        index_name="procurement-index",
        number_of_results=100,
    )
    try:
        context_docs = await get_context_data_async(retriever, state.question)
    except Exception:
        context_docs = []

    messages = [
        {"role": "system", "content": "You are the procurement agent."},
        {"role": "user", "content": state.question},
    ]
    messages.append({"role": "user", "content": "Agent: Procurement"})
    if state.category:
        messages.append({"role": "user", "content": f"Category: {state.category}"})
    if context_docs:
        messages.append({"role":"user", "content": context_docs[0].page_content})

    result = await llm.ainvoke(messages)
    from pydantic import BaseModel
    class GraphOutput(BaseModel):
        response: str
    return GraphOutput(response=result.content)


In [ ]:
# Added route_by_category(state) to return the next node key

def route_by_category(state: GraphState) -> str:
    if state.category and state.category.lower() == "procurement":
        return "procurement"
    return "policy"


In [ ]:
# Rewired graph: START -> supervisor -> (policy | procurement) -> END (demo wiring)
from importlib import reload
import main as main_module
reload(main_module)
policy_node = main_module.policy_node

from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel

class GraphOutput(BaseModel):
    response: str

builder_demo = StateGraph(GraphState, output=GraphOutput)
builder_demo.add_node("supervisor", supervisor_node)
builder_demo.add_node("policy", policy_node)
builder_demo.add_node("procurement", procurement_node)

builder_demo.add_edge(START, "supervisor")
builder_demo.add_conditional_edges(
    "supervisor",
    route_by_category,
    {
        "policy": "policy",
        "procurement": "procurement",
    },
)

builder_demo.add_edge("policy", END)
builder_demo.add_edge("procurement", END)

demo_graph = builder_demo.compile()


----

### Sync & visualise the Agent

In [ ]:
# Update the main.py
from pathlib import Path
src = Path("resources/main2_multiple_agents.py")
dst = Path("main.py")
dst.write_text(src.read_text(), encoding="utf-8")
print("Synchronized main.py from resources/main2_multiple_agents.py")

In [ ]:
# Re-initialize the project to make sure all file are up to date
!uipath init

In [ ]:
# See the Mermaid diagram
from IPython.display import display, Markdown

with open("agent.mermaid", "r") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

**Test Procurement flow**

In [ ]:
%%writefile input.json
{
  "question": "Can I buy an OpenAI subscription for company use?"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![UiPath_Supervisor_Procurement_Test](resources/UiPath_Supervisor_Procurement_Test.png)

**Test Policy flow**

In [ ]:
%%writefile input.json
{
  "question": "What is the PTO policy for taking 3 days off?"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![UiPath_Supervisor_Policy_Test](resources/UiPath_Supervisor_Policy_Test.png)

## **Control**. Supports deterministic agent logic and full flexibility

- Blends coded steps with LLM-powered, nondeterministic decision points
- Full flexibility. If you can code it you can do it


**Deterministic agent logic**

Add a **deterministic** validation step on **procurement path**

- **GraphState**: added optional `email: str | None` and `code: str | None`.
- **New node** `verify_credentials` on the Procurement path (routing: supervisor → verify_credentials → procurement)
- **Validators**: added `is_valid_company_email(email)` and `is_valid_code(code)`.
- **Procurement node updates**: validate optional inputs; fail fast if missing/invalid; proceed if valid (and include verification context in messages).


----

### Sync & visualise the Agent

In [ ]:
# Update main.py with Validation node

from pathlib import Path
src = Path("resources/main3_validation.py")
dst = Path("main.py")
dst.write_text(src.read_text(), encoding="utf-8")
print("Synchronized main.py from resources/main3_validation.py")

In [ ]:
# Re-initialize the project to make sure all file are up to date
!uipath init

In [ ]:
# See the Mermaid diagram
from IPython.display import display, Markdown

with open("agent.mermaid", "r") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


**Test Procurement + No Credentials Flow**

In [ ]:
%%writefile input.json
{
  "question": "Can I buy an OpenAI subscription for company use?"
}

In [ ]:
!uipath run agent --file input.json


**Expected output**

![Main3_Failed_Procurement_Path](resources/Main3_Failed_Procurement_Path.png)

**Test Procurement + Valid Credentials Flow**

In [ ]:
%%writefile input.json
{
  "question": "Can I buy an OpenAI subscription for company use?",
  "email": "eusebiu.jecan@uipath.com",
  "code": "4827"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![Main3_Success_Procurement_Path_1](resources/Main3_Success_Procurement_Path_1.png)
![Main3_Success_Procurement_Path_2](resources/Main3_Success_Procurement_Path_2.png)



## Reusability & Extensibility

- Reuse and tailor across multiple scenarios
- Iteratively evolve agents to improve accuracy and expand capabilities

Reuse the Supervisor node and existing pattern to add a dedicated HR Agent to help with HR Specific tasks

- **Supervisor**: added “HR” to categories and updated the classification prompt to include HR.
- **New node hr on the HR path** (routing: supervisor → hr → END).
- **Local catalog**: reads hr_tools.json (root) or resources/hr_tools.json; falls back to a built-in default mapping if missing.
- **Deterministic output**: always returns Tool selected for the ask is: <tool>; defaults to none on parsing failure.

----

### Sync & visualise the Agent

In [ ]:
# Update main.py with HR Agent

from pathlib import Path
src = Path("resources/main4_hr.py")
dst = Path("main.py")
dst.write_text(src.read_text(), encoding="utf-8")
print("Synchronized main.py from resources/main4_hr.py")

In [ ]:
# Re-initialize the project to make sure all file are up to date
!uipath init

In [ ]:
# See the Mermaid diagram
from IPython.display import display, Markdown

with open("agent.mermaid", "r") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

In [ ]:
%%writefile input.json
{
  "question": "Schedule an HR screening for Eusebiu Jecan",
  "email": "eusebiu.jecan@uipath.com",
  "code": "4827"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![Main4_Output](resources/Main4_Output.png)

## **Customization**. Enables niche and highly specialized scenarios​

- Integration with bespoke software solutions
- Build and use coded MCP Servers

We can add granular customisation to:

- **Use a beskope, local DB** to check if the user is authorised to use the HR Agent as it might return sensitive info with limited access (as opposed to Policy that can be available for everyone in the company 

----

### Sync & visualise the Agent

In [ ]:
# Update main.py with Permission Check for HR

from pathlib import Path
src = Path("resources/main5_permission_check_hr.py")
dst = Path("main.py")
dst.write_text(src.read_text(), encoding="utf-8")
print("Synchronized main.py from resources/main5_permission_check_hr.py")

In [ ]:
# Re-initialize the project to make sure all file are up to date
!uipath init

In [ ]:
# See the Mermaid diagram
from IPython.display import display, Markdown

with open("agent.mermaid", "r") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

**The allowd list includes eusebiu.jecan@uipath.com, edit in code other/your e-mail address to test**

In [ ]:
# Create a local HR authorization file (allow-list)
import json, os, textwrap

auth = {
    "allowed_emails": [
        "eusebiu.jecan@uipath.com",
        # Add others here...
    ]
}

# Write to project root
with open("hr_auth.json", "w", encoding="utf-8") as f:
    json.dump(auth, f, indent=2)

print("Created hr_auth.json with allow-list:", auth["allowed_emails"])

In [ ]:
%%writefile input.json
{
  "question": "Schedule an HR screening for Eusebiu Jecan",
  "email": "eusebiu.jecan@uipath.com",
  "code": "4827"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![Authorized HR screening](resources/Main5_Authorised.png)


In [ ]:
%%writefile input.json
{
  "question": "Schedule an HR screening for Eusebiu Jecan",
  "email": "somenone@uipath.com",
  "code": "4827"
}

In [ ]:
!uipath run agent --file input.json

**Expected output**

![Authorized HR screening](resources/Main5_Not_Authorised.png)

## Tailored & deterministic Human-in-the-loop

### Sync & visualise the Agent

In [ ]:
# Update main.py with Human in the loop

from pathlib import Path
src = Path("resources/main6_HITL.py")
dst = Path("main.py")
dst.write_text(src.read_text(), encoding="utf-8")
print("Synchronized main.py from resources/main6_HITL.py")

**Set the Human in the loop assignee email**

In [ ]:
# Re-initialize the project to make sure all file are up to date

!uipath init

In [ ]:
# Visualize the current graph (Mermaid)
from IPython.display import display, Markdown

with open("agent.mermaid", "r", encoding="utf-8") as f:
    mermaid_code = f.read()

display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

In [ ]:
%%writefile input.json
{
  "question": "I met a new prospect at a conference, I need to add him to the CRM",
  "email": "eusebiu.jecan@uipath.com",
  "code": "4827"
}

In [ ]:
# Ensure only the correct UIPATH_URL is enabled in .env
lines = []
with open(".env", "r", encoding="utf-8") as f:
    for line in f:
        if line.strip().startswith("UIPATH_URL="):
            # Comment out all UIPATH_URL lines
            lines.append(f"# {line.lstrip('#').strip()}\n")
        else:
            lines.append(line)
# Add the correct UIPATH_URL at the end
lines.append("UIPATH_URL=https://cloud.uipath.com/722912cc-2151-46d4-812f-dd2d575f0739/e12bc62d-a788-4bfd-a29f-74b907ec6ca5\n")
with open(".env", "w", encoding="utf-8") as f:
    f.writelines(lines)
print("Updated .env with the correct UIPATH_URL")


In [ ]:
# Set the HITL assignee email 
HITL_ASSIGNEE_EMAIL = "allowed.email@provider.com"  # <--- Change this value with teh e-mail you signed up within UiPath

from pathlib import Path
import re

main_path = Path("main.py")
lines = main_path.read_text(encoding="utf-8").splitlines()

# Find the line with 'assignee=' and replace the email
for i, line in enumerate(lines):
    if "assignee=" in line:
        lines[i] = re.sub(
            r'assignee\s*=\s*["\'].*?["\']',
            f'assignee=\"{HITL_ASSIGNEE_EMAIL}\"',
            line
        )
        break

main_path.write_text("\n".join(lines), encoding="utf-8")
print(f"Updated HITL assignee email in main.py to: {HITL_ASSIGNEE_EMAIL}")

In [ ]:
!uipath run agent --file input.json

In [ ]:
!uipath run agent --resume 

## Local development
After installing the `uipath-dev` package, you can open your terminal and run the command `uipath dev` to see your agent in action, including logs and traces.

In [ ]:
%pip install uipath-dev

# Evals
# MCP
# Vibe Coding